In [24]:
%cd /home/Reguformer/notebooks

/home/Reguformer/notebooks


In [25]:
import datetime
import json
import os
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn

from copy import deepcopy
from xgboost import XGBClassifier
from torch.utils.data import TensorDataset


warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

from trans_oil_gas import utils_emb, utils_fix_seeds, utils_model, utils_dataset

Choose GPU device if it is available.

In [26]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
gpu = [0] if 'cuda' in device.type else 0
print(device, gpu)

cpu 0


Fix all possible seeds for results reproducibility.

In [27]:
utils_fix_seeds.fix_seeds(device=device)

# Read data

Reading of already preprocessed data (here synthetic dataset with the same columns as in the original one is presented).

In [28]:
df = pd.read_csv('../data/synthetic_well_log_data.csv')

# Load models

In [29]:
# results_len = 5000
results_len = 500
slice_len = 100
input_size = 4

In [30]:
path_to_models = "./saves_all_models/"

In [31]:
shared_fixed_params = {
    "enc_in": 4,
    "distil": None,
    "device": device,
    "output_attention": False,
    "n_seq": 100
}
fixed_params_tr = {
    "attn": "full",
    "activation": "relu",
}
fixed_params_reguformer = {
    "attn": "prob",
    "activation": "gelu",
}

In [32]:
path_to_save = './saves_emb_quality-classification/'
path_to_logs = './logs_emd_quality-classification'

if not os.path.exists(path_to_save):
    os.mkdir(path_to_save)

if not os.path.exists(path_to_logs):
    os.mkdir(path_to_logs)

In [33]:
batch_size = 64
# epochs = 100
epochs = 2

# Generate dataset

In [34]:
_, slice_dataset = utils_dataset.generate_datasets(
    df, df,
    well_column="WELLNAME",
    slice_len=slice_len, 
    path_to_saves=path_to_save,
    results_len_train=0,
    results_len_test=results_len,
)

# Obtain embeddings and classify them

In [35]:
for loss_type in ["siamese", "triplet"]:
    for model_type in [
        "None",
        "topQ", "randQ", 
        "topK", "randK", 
        "topQ_topK", "topQ_randK", 
        "randQ_topK", "randQ_randK",
    ]:
        print(loss_type, model_type)
        with open(os.path.join(path_to_models, "best_params_{}_reguformer_{}.json".format(loss_type, model_type)), "r") as f:
            best_params = json.load(f)
        
        if "None" in model_type:
            fixed_params = deepcopy(fixed_params_tr)
            enc_type = "transformer"
            reg_type = ""
        else:
            fixed_params = deepcopy(fixed_params_reguformer)
            enc_type = "reguformer"
            reg_type = "_" + model_type

        if "siamese" in loss_type:
            model = utils_model.SiameseArchitecture(
                encoder_type="reguformer", 
                fc_hidden_size=64, 
                **shared_fixed_params, 
                **fixed_params,
                **best_params,
            ).float()

        elif "triplet" in loss_type:
            model = utils_model.TripletArchitecture(
                encoder_type="reguformer", 
                **shared_fixed_params, 
                **fixed_params,
                **best_params,
            ).float()

        model.load_state_dict(torch.load(os.path.join(path_to_models, "res_{}_{}{}.pth".format(loss_type, enc_type, reg_type)), map_location=device))

        model.eval()
        embeddings, wells = utils_emb.get_embeddings(
            model,
            model_type,
            slice_dataset,
            slice_len=slice_len,
            input_size=df.shape[1]-1
        )
        embeddings = embeddings[:, None, :]
        
        le = LabelEncoder()
        wells_tr = le.fit_transform(wells)
        wells_tr = torch.tensor(wells_tr)

        train_wells = []
        test_wells = []
        train, test = torch.tensor([]), torch.tensor([])
        
        for well in np.unique(wells_tr):
            embs = embeddings[wells_tr == well]
            
            train = torch.cat(
                [
                    train, 
                    embs[:len(embs) // 2 + 1].detach()
                ], dim=0
            )
            test = torch.cat(
                [
                    test, 
                    embs[len(embs) // 2 + 1:].detach()
                ], dim=0
            )
            train_wells.extend([well for _ in range(0, len(embs) // 2 + 1)])
            test_wells.extend([well for _ in range(0, len(embs) - len(embs) // 2 - 1)])
        
        input_size = embeddings.shape[-1]
        output_size = len(torch.unique(wells_tr))

        print("XGB training ...")
        results_xgb = []
        model = XGBClassifier()

        model.fit(
            train.detach().cpu().numpy()[:, 0, :],
            train_wells
        )

        y_preds = model.predict_proba(test.detach().cpu().numpy()[:, 0, :])

        acc, roc_auc, pr_auc, precision, recall, f1 = utils_emb.calculate_metrics_for_final(
            target=torch.tensor(test_wells),
            y_pred=torch.tensor(y_preds)
        )
        results_xgb.append((model_type, acc, precision, recall, f1, roc_auc, pr_auc))
        
        results_xgb_df = pd.DataFrame(results_xgb, columns=["Model type", "Accuracy", "ROC AUC", "PR AUC", "Precision", "Recall", "F1"])
        results_xgb_df.to_csv(os.path.join(path_to_save, 'emb_quality_xgb_{}_{}.csv'.format(loss_type, model_type)))

        print("1 lin.layer training...")
        results_1_layer = []
        model = nn.Linear(input_size, output_size)

        # model, logger, (acc, roc_auc, pr_auc, precision, recall, f1) = utils_emb.train_test_linear_classifier(
        #     train,
        #     test,
        #     train_wells,
        #     test_wells,
        #     model,
        #     model_type,
        #     batch_size=batch_size,
        #     epochs=epochs,
        #     gpu=device.index,
        #     log_dir=path_to_logs,
        # )
        train_dataset = TensorDataset(train, torch.tensor(train_wells))
        test_dataset = TensorDataset(test, torch.tensor(test_wells))

        clf = utils_emb.ClassificationModel(
            model=model,
            train_data=train_dataset,
            test_data=test_dataset,
            n_classes=len(np.unique(train_wells)),
            batch_size=batch_size,
        )

        current_time = datetime.datetime.now().strftime("%m%d%Y_%H:%M:%S")
        add_name = "_transformer_" if "None" in model_type else "_reguformer_"
        experiment_name = loss_type + add_name + model_type + "_" +  "lin_layer" + "_" + current_time

        logger = pl.loggers.TensorBoardLogger(save_dir=path_to_logs, name=experiment_name)

        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            monitor="val_loss",
            dirpath=f"{path_to_logs}/{experiment_name}/",
            filename="{epoch:02d}-{val_loss:.3f}",
            mode="min",
        )

        trainer = pl.Trainer(
            max_epochs=epochs,
            gpus=gpu,  # set 1 to use one GPU
            benchmark=True,
            check_val_every_n_epoch=1,
            logger=logger,
            callbacks=[
                checkpoint_callback,
                pl.callbacks.early_stopping.EarlyStopping(
                    monitor="val_loss", patience=40, mode="min"
                ),
            ],
        )
        trainer.fit(clf)
        y_pred = model(test)
        output = (
            y_pred.detach()
            .numpy()
            .reshape(y_pred.detach().numpy().shape[0], y_pred.detach().numpy().shape[-1])
        )
        acc, roc_auc, pr_auc, precision, recall, f1 = utils_emb.calculate_metrics_for_final(
            target=torch.tensor(test_wells), y_pred=torch.tensor(output)
        )
        print(
            "Accuracy: {}, ROC AUC: {}, PR AUC: {}, Precision: {}, Recall: {}, F1: {}".format(
                np.round(acc, 4),
                np.round(roc_auc, 4),
                np.round(pr_auc, 4),
                np.round(precision, 4),
                np.round(recall, 4),
                np.round(f1, 4),
            )
        )
        results_1_layer.append((model_type, acc, precision, recall, f1, roc_auc, pr_auc))

        results_1_layer_df = pd.DataFrame(results_1_layer, columns=["Model type", "Accuracy", "ROC AUC", "PR AUC", "Precision", "Recall", "F1"])
        results_1_layer_df.to_csv(os.path.join(path_to_save, 'emb_quality_1_lin_layer_{}_{}.csv'.format(loss_type, model_type)))

        print("FCNN training...")
        results_multi_layer = []
        hidden_sizes = [64, 128]

        # Build a feed-forward network
        model = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], output_size),
        )
        # model, logger, (acc, roc_auc, pr_auc, precision, recall, f1) = utils_emb.train_test_linear_classifier(
        #     train,
        #     test,
        #     train_wells,
        #     test_wells,
        #     model,
        #     model_type,
        #     batch_size=batch_size,
        #     epochs=epochs,
        #     gpu=device.index,
        #     log_dir=path_to_logs,
        # )
        # train_dataset = TensorDataset(train, torch.tensor(train_wells))
        # test_dataset = TensorDataset(test, torch.tensor(test_wells))

        clf = utils_emb.ClassificationModel(
            model=model,
            train_data=train_dataset,
            test_data=test_dataset,
            n_classes=len(np.unique(train_wells)),
            batch_size=batch_size,
        )

        current_time = datetime.datetime.now().strftime("%m%d%Y_%H:%M:%S")
        experiment_name = loss_type + add_name + model_type + "_" +  "fcnn" + "_" + current_time

        logger = pl.loggers.TensorBoardLogger(save_dir=path_to_logs, name=experiment_name)

        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            monitor="val_loss",
            dirpath=f"{path_to_logs}/{experiment_name}/",
            filename="{epoch:02d}-{val_loss:.3f}",
            mode="min",
        )

        trainer = pl.Trainer(
            max_epochs=epochs,
            gpus=gpu,  # set 1 to use one GPU
            benchmark=True,
            check_val_every_n_epoch=1,
            logger=logger,
            callbacks=[
                checkpoint_callback,
                pl.callbacks.early_stopping.EarlyStopping(
                    monitor="val_loss", patience=40, mode="min"
                ),
            ],
        )
        trainer.fit(clf)
        y_pred = model(test)
        output = (
            y_pred.detach()
            .numpy()
            .reshape(y_pred.detach().numpy().shape[0], y_pred.detach().numpy().shape[-1])
        )
        acc, roc_auc, pr_auc, precision, recall, f1 = utils_emb.calculate_metrics_for_final(
            target=torch.tensor(test_wells), y_pred=torch.tensor(output)
        )
        print(
            "Accuracy: {}, ROC AUC: {}, PR AUC: {}, Precision: {}, Recall: {}, F1: {}".format(
                np.round(acc, 4),
                np.round(roc_auc, 4),
                np.round(pr_auc, 4),
                np.round(precision, 4),
                np.round(recall, 4),
                np.round(f1, 4),
            )
        )
        results_multi_layer.append((model_type, acc, precision, recall, f1, roc_auc, pr_auc))

        results_multi_layer_df = pd.DataFrame(results_multi_layer, columns=["Model type", "Accuracy", "ROC AUC", "PR AUC", "Precision", "Recall", "F1"])
        results_multi_layer_df.to_csv(os.path.join(path_to_save, 'emb_quality_fc_nn_{}_{}.csv'.format(loss_type, model_type)))

24it [00:00, 234.18it/s]

siamese None


500it [00:02, 233.75it/s]


XGB training ...
[17:46:53] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | Linear           | 68    
1 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
68        Trainable params
0         Non-trainable params
68        Total params
0.000     Total estimated model params size (MB)


...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

mean_accuracy = 0.5
mean_roc_auc = 0.9518947277141866
mean_pr_auc = 0.8555946528786217


Validating: 0it [00:00, ?it/s]

mean_accuracy = 0.75
mean_roc_auc = 0.9977020464793617
mean_pr_auc = 0.9931949132833896
Accuracy: 0.75, ROC AUC: 0.9977, PR AUC: 0.9932, Precision: 0.625, Recall: 0.75, F1: 0.6818


In [36]:
# the same for Performer